In [2]:
import numpy as np
import h5py
import pandas as pd
import time
outpath = '/project/chihway/raulteixeira/data'

def flux2mag(flux):
    '''
    Converts fluxes to magnitudes
    '''
    return -2.5 * np.log10(flux) + 30

In [1]:
keys = ['FLAGS_FOREGROUND', 'ID', 'Truth_ind', 'badfrac', 'd_arcsec', 'd_contam_arcsec', 'dec',\
        'detected', 'id', 'inj_class', 'mcal_T_err_noshear', 'mcal_T_noshear', 'mcal_T_ratio_noshear',\
        'mcal_s2n_noshear', 'mcal_s2n_r_noshear', 'ra', 'tilename', 'true_dec', 'true_ra', 'mcal_flags']

df = pd.DataFrame()
with h5py.File('/project/chihway/dhayaa/DECADE/BalrogOfTheDECADE_20231120.hdf5') as f:
    flux_r, flux_i, flux_z = np.array(f['mcal_flux_noshear']).T
    flux_err_r, flux_err_i, flux_err_z = np.array(f['mcal_flux_err_noshear']).T
    g1, g2  = np.array(f['mcal_g_noshear']).T
    
    df['mcal_flux_r_noshear'] = flux_r
    df['mcal_flux_r_err_noshear']= flux_err_r
    df['mcal_flux_i_noshear'] = flux_i
    df['mcal_flux_i_err_noshear']= flux_err_i
    df['mcal_flux_z_noshear'] = flux_z
    df['mcal_flux_z_err_noshear']= flux_err_z
    
    df['g1'] = g1
    df['g2'] = g2
    for key in keys:
        df[key]= np.array(f[key])

# df.loc[df['detected'].astype(bool)&(df.FLAGS_FOREGROUND.values==0)]

NameError: name 'pd' is not defined

In [17]:
df.to_hdf('%s/BalrogoftheDECADE_112523.hdf5'%outpath, key='df')

# np.savez('%s/BalrogoftheDECADE_Mask112523.npz'%outpath, df['detected'].astype(bool).values)

/tmp/jobs/31336570/ipykernel_247322/2327421322.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->bytes,key->block5_values] [items->Index(['tilename'], dtype='object')]

  df.to_hdf('%s/BalrogoftheDECADE_112523.hdf5'%outpath, key='df')


In [31]:
# mask = np.load('%s/BalrogoftheDECADE_Mask112523.npz'%outpath)

In [3]:
df = pd.read_hdf('%s/BalrogoftheDECADE_112523.hdf5'%outpath, key='df')

In [9]:
!ls /project/chihway/raulteixeira/data/BalrogoftheDECADE_112523.hdf5 -lht

-rw-rw-r-- 1 raulteixeira pi-chihway 1.6G Nov 26 00:53 /project/chihway/raulteixeira/data/BalrogoftheDECADE_112523.hdf5


In [22]:
t0 = time.time()

fluxcols    = ['mcal_flux_r_noshear', 'mcal_flux_i_noshear', 'mcal_flux_z_noshear']
fluxerrcols = ['mcal_flux_r_err_noshear', 'mcal_flux_i_err_noshear', 'mcal_flux_z_err_noshear']

flux_r, flux_i, flux_z = (df[fluxcol] for fluxcol in fluxcols)
flux_err_r, flux_err_i, flux_err_z = (df[fluxerrcol] for fluxerrcol in fluxerrcols)

# flux_err_r, flux_err_i, flux_err_z = np.array(f['mcal_flux_err_noshear']).T
mag_r = flux2mag(flux_r)
mag_i = flux2mag(flux_i)
mag_z = flux2mag(flux_z)

mcal_pz_mask = ((mag_i < 23.5) & (mag_i > 18) & 
                (mag_r < 26)   & (mag_r > 15) & 
                (mag_z < 26)   & (mag_z > 15) & 
                (mag_r - mag_i < 4)   & (mag_r - mag_i > -1.5) & 
                (mag_i - mag_z < 4)   & (mag_i - mag_z > -1.5))

SNR     = df['mcal_s2n_noshear']
T_ratio = df['mcal_T_ratio_noshear']
T       = df['mcal_T_noshear']
flags   = df['mcal_flags']

fg = df['FLAGS_FOREGROUND']

g1, g2  = df['g1'], df['g2']

#Metacal cuts based on DES Y3 ones (from here: https://des.ncsa.illinois.edu/releases/y3a2/Y3key-catalogs)
SNR_Mask   = (SNR > 10) & (SNR < 1000)
Tratio_Mask= T_ratio > 0.5
T_Mask     = T < 10
Flag_Mask  = flags == 0
#SG_Mask = (sg>=4)
FG_Mask = (fg==0)

Other_Mask = np.invert((T > 2) & (SNR < 30)) & np.invert((np.log10(T) < (22.25 - mag_r)/3.5) & (g1**2 + g2**2 > 0.8**2))

Mask = mcal_pz_mask & SNR_Mask & Tratio_Mask & T_Mask & Flag_Mask & Other_Mask & FG_Mask #& SG_Mask

np.savez('/project/chihway/raulteixeira/data/Balrog_20231125_MASK_GOLD+WL+FG.npz', Mask)

print('time cut:', time.time()-t0, 'seconds')

time cut: 1.6094179153442383 seconds


In [5]:
np.load('/project/chihway/raulteixeira/data/Balrog_20231125_MASK_GOLD+WL+FG.npz')['arr_0'].sum()

1312588

In [31]:
Mask.sum()

1312588

In [33]:
fp_mask = (df['true_ra'] < 180) & (df['true_dec'] > -25)

In [34]:
fp_mask

5309281

In [46]:
good_DF_ids = np.unique(df['ID'][fp_mask&Mask&class_mask])

In [58]:
good_DF_ids.size

154886

In [44]:
raw_DF['ID']

0          692581195
1          692581196
2          692581197
3          692581243
4          692581244
             ...    
2826983    698430493
2826984    698430494
2826985    698430540
2826986    698430541
2826987    698430542
Name: ID, Length: 2826988, dtype: int64

In [75]:
wrg3e = (Mask)[df.detected&fp_mask]

In [77]:
wrg3e.mean()

0.28153078318391855

In [54]:
clean_DF = pd.read_csv('/project2/chihway/raulteixeira/data/deepfields_clean.csv.gz')
alex_DF = pd.read_csv('/project/chihway/raulteixeira/data/deepfields_with_redshifts.csv.gz')

In [55]:
good_DF = clean_DF[np.isin(clean_DF['ID'], good_DF_ids)]

In [56]:
good_DF = good_DF.merge(alex_DF[['ID', 'Z']], on='ID', how='left')

In [80]:
good_DF.to_hdf('%s/deep_Balrog_DR3_1_1_fp_GOLD_WL_FG.hdf5'%outpath, key='df')

In [ ]:
good_DF = 

In [59]:
np.isfinite(good_DF['Z']).sum()

55084

In [36]:
(fp_mask&Mask&class_mask).sum()

834737

In [9]:
df.columns

Index(['mcal_flux_r_noshear', 'mcal_flux_r_err_noshear', 'mcal_flux_i_noshear',
       'mcal_flux_i_err_noshear', 'mcal_flux_z_noshear',
       'mcal_flux_z_err_noshear', 'g1', 'g2', 'FLAGS_FOREGROUND', 'ID',
       'Truth_ind', 'badfrac', 'd_arcsec', 'd_contam_arcsec', 'dec',
       'detected', 'id', 'inj_class', 'mcal_T_err_noshear', 'mcal_T_noshear',
       'mcal_T_ratio_noshear', 'mcal_s2n_noshear', 'mcal_s2n_r_noshear', 'ra',
       'tilename', 'true_dec', 'true_ra'],
      dtype='object')

In [ ]:
shorter_cols = ['mcal_flux_r_noshear', 'mcal_flux_r_err_noshear', 'mcal_flux_i_noshear',
                'mcal_flux_i_err_noshear', 'mcal_flux_z_noshear', 'mcal_flux_z_err_noshear', 
                'ID', 'id','tilename']

In [15]:
class_mask = df['detected'].astype(bool).values & (df['d_contam_arcsec'].values>1.5)

In [18]:
wide_cells.shape

(4935450,)

In [20]:
df['wide_cells'] = np.nan
df['wide_cells'][class_mask] = wide_cells

/tmp/jobs/31339947/ipykernel_297192/1151916408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wide_cells'][class_mask] = wide_cells


In [22]:
df.to_hdf('%s/BalrogoftheDECADE_112523_wide_cells.hdf5'%outpath, key='df')

/tmp/jobs/31339947/ipykernel_297192/2728549286.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->bytes,key->block5_values] [items->Index(['tilename'], dtype='object')]

  df.to_hdf('%s/BalrogoftheDECADE_112523_wide_cells.hdf5'%outpath, key='df')


In [16]:
np.savez('%s/BalrogoftheDECADE_Mask_DET_CONTAM_1dot5_112523.npz'%outpath, class_mask)
df[shorter_cols][class_mask].to_csv('%s/BalrogoftheDECADE_112523_MASKED_DET_CONTAM_1dot5_fluxes+errs+ID+id+tilename.csv.gz'%outpath)

In [ ]:
df[~df['detected'].astype(bool).values].sum()

In [ ]:
np.rec.array(df[~df['detected'].astype(bool).values])

In [18]:
!ls -lht /project/chihway/raulteixeira/data/BalrogoftheDECADE_112523.hdf5

-rw-rw-r-- 1 raulteixeira pi-chihway 1.6G Nov 26 00:53 /project/chihway/raulteixeira/data/BalrogoftheDECADE_112523.hdf5


In [6]:
n_deep = 48
outpath = '/project/chihway/raulteixeira/data'

balrog_det_DF = pd.read_hdf('%s/deep_Balrog_DR3_1_1_fp_GOLD_WL_FG.hdf5'%outpath, key='df')

In [7]:
balrog_det_DF

,ID,RA,DEC,KNN_CLASS,BDF_FLUX_DERED_CALIB_U,BDF_FLUX_DERED_CALIB_G,BDF_FLUX_DERED_CALIB_R,BDF_FLUX_DERED_CALIB_I,BDF_FLUX_DERED_CALIB_Z,BDF_FLUX_DERED_CALIB_J,...,BDF_FLUX_DERED_CALIB_KS,BDF_FLUX_ERR_DERED_CALIB_U,BDF_FLUX_ERR_DERED_CALIB_G,BDF_FLUX_ERR_DERED_CALIB_R,BDF_FLUX_ERR_DERED_CALIB_I,BDF_FLUX_ERR_DERED_CALIB_Z,BDF_FLUX_ERR_DERED_CALIB_J,BDF_FLUX_ERR_DERED_CALIB_H,BDF_FLUX_ERR_DERED_CALIB_KS,Z
0,692581197,52.919675,-28.867296,1.0,204.186270,259.924072,640.058928,923.667161,990.715877,1470.225204,...,2012.706674,14.830872,6.298630,11.451556,16.559125,29.037093,38.015369,84.564620,56.049189,NaN
1,692581245,52.806489,-28.867570,1.0,482.933499,471.033683,666.071153,1136.945773,1345.197659,1742.870626,...,1956.258491,17.874394,7.675500,11.668144,18.085892,31.539036,42.248013,65.293374,58.042663,NaN
2,692581291,52.847828,-28.869228,1.0,1094.144134,1352.384103,2619.408382,4579.472224,5198.553619,7106.616134,...,9941.873784,28.111577,11.526915,19.684373,31.248676,52.726051,54.929058,83.638143,63.378619,NaN
3,692581293,52.856823,-28.870751,1.0,1986.774586,2948.795729,5489.816636,6215.050206,7144.472830,8527.280981,...,10108.686762,36.442989,16.605563,29.040082,37.448289,67.742527,46.799034,109.493905,68.978164,NaN
4,692581435,52.813987,-28.872141,1.0,1707.829727,2646.956330,4638.808679,5388.150997,6078.494828,7333.607844,...,8987.379186,25.853381,12.191459,20.531253,26.672065,46.166829,24.039285,53.206740,36.168111,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144041,698431163,36.978001,-4.969034,1.0,191.622830,656.359471,2309.533102,3801.424896,4924.267036,8947.095761,...,16173.745160,20.108306,6.880690,11.452194,19.616352,31.847288,36.694250,43.902839,62.018809,NaN
144042,698431259,36.954328,-4.969575,1.0,120.250526,216.091623,506.742228,825.888467,965.745010,1440.678393,...,2379.999600,19.957054,6.650911,9.950356,17.043753,27.546567,47.713860,56.839087,69.299714,NaN
144043,698431931,36.889803,-4.975466,1.0,347.428774,524.071173,601.148175,624.480941,659.387098,1004.714485,...,1679.988042,18.309016,8.481602,10.596290,15.401787,26.191373,14.726907,20.140130,26.876241,NaN
144044,698432027,37.016463,-4.977824,1.0,802.635318,1458.256794,2833.101547,3339.585189,3624.605553,4458.817035,...,5281.051492,27.929286,11.570527,18.455662,27.335210,44.253162,23.356293,33.124416,48.277439,NaN


In [5]:
bands = list('UGRIZJH')+['KS']

fluxes_d = np.array([balrog_det_DF[f'BDF_FLUX_DERED_CALIB_{band}'].values for band in bands]).T
fluxerrs_d = np.array([balrog_det_DF[f'BDF_FLUX_ERR_DERED_CALIB_{band}'].values for band in bands]).T

def flux2mag(flux):
    return -2.5 * np.log10(flux) + 30

# cols = [f'flux_{band.lower()}' for band in bands]+[f'flux_err_{band.lower()}' for band in bands]
# df = pd.DataFrame(np.hstack([fluxes_d, fluxerrs_d]), columns=cols)

# mags_d = np.zeros((len(df),len(bands)))
# magerrs_d = np.zeros((len(df),len(bands)))

# for i,band in enumerate(bands):
#     print(i,band)
#     mags_d[:,i] = flux2mag(fluxes_d[:,i])

# colors = np.zeros((len(fluxes_d),len(bands)-1))
# for i in range(len(bands)-1):
#     colors[:,i] = mags_d[:,i] - mags_d[:,i+1]

# normal_colors = np.mean(colors > -1, axis=1) == 1
# normal_colors.sum()

# df = df[normal_colors]

# #mask faint objects, i < 25
balrog_det_DF = balrog_det_DF[flux2mag(balrog_det_DF.BDF_FLUX_DERED_CALIB_I.values) < 25]
print('length of dataframe and number of redshift samples:', len(balrog_det_DF), np.sum(np.isfinite(balrog_det_DF['Z'])))

length of dataframe and number of redshift samples: 144046 55084


In [4]:
df = pd.read_csv('%s/BalrogoftheDECADE_112523_MASKED_DET_CONTAM_1dot5_fluxes+errs+ID+id+tilename.csv.gz'%outpath)

wide_cells = np.load("%s/som_BalrogoftheDECADE_112523_26x26.npz"%outpath)['cells']
df['wide_cells']=wide_cells

In [11]:
np.histogram(df['wide_cells'])

(array([1590774,  291264,  254521,  342487,  295201,  512856,  447482,
         367775,  298578,  534512]),
 array([  0. ,  67.5, 135. , 202.5, 270. , 337.5, 405. , 472.5, 540. ,
        607.5, 675. ]))

In [5]:
balrog_full = pd.read_hdf('%s/BalrogoftheDECADE_112523.hdf5'%outpath, key='df')

In [ ]:
balrog_full.merge(df, )